# Mapping Epicollect points in Hillside Woods

Our goal is to display the user's current location, using a phone GPS point, simultaneously alongside EpiCollect5 data for the project (https://five.epicollect.net/project/hillside-woods-restoration-project).  

We can do this using a Python package to collect data using the EpiCollect5 API, using the `folium` mapping tool (which wraps leaflet.js), and using the LocateControl functionality in leaflet.js (code here: https://github.com/python-visualization/folium/blob/master/folium/plugins/locate_control.py)

### installs (run these in terminal)

In [1]:
# pip install pyepicollect
# conda install folium -c conda-forge

### imports

In [2]:
# package for getting EpiCollect data
import pyepicollect as pyep
# package for mapping
import folium
# tool for adding current location
from folium.plugins import LocateControl
# linear algebraic tools
import numpy as np

# 1) Access and format EpiCollect5 data using `pyepicollect`

In [3]:
# load entries -- set the per_page value high so that we get em all
entries = pyep.api.get_entries('hillside-woods-restoration-project',per_page=10000)

In [4]:
# save list of lats
lats = [i['5_Location']['latitude'] for i in entries['data']['entries']]

# save list of lons
lons = [i['5_Location']['longitude'] for i in entries['data']['entries']]

In [5]:
# zip em together
lat_lon_pairs = [list(i) for i in zip(lats,lons)]

# look at the first several:
lat_lon_pairs[:10]

[[40.996061, -73.871626],
 [40.996034, -73.87145],
 [40.995865, -73.871419],
 [40.995363, -73.871993],
 [40.995387, -73.871952],
 [40.995362, -73.871958],
 [40.995369, -73.871935],
 [40.995629, -73.872033],
 [40.995688, -73.872089],
 [40.995568, -73.871644]]

# 2) Define a map with `folium`

## Map:

#### Start by defining map parameters:

In [6]:
# give the map a general starting location (middle of hillside woods)
starting_location = [40.997895, -73.867473]

# give the map a starting zoom
zoom_start = 16

# give the map a maximum zoom -- higher number allows farther zoom, but makes things slower...
max_zoom = 22

# maybe giving a min_zoom will speed up rendering?
min_zoom = 0

#### Now define the map and locator:

In [7]:
# define a simple map
m = folium.Map(starting_location,
               zoom_start=zoom_start, 
               max_zoom=max_zoom,
               min_zoom=min_zoom)

# turn on the gps locator
LocateControl(strings={"title": "See your current location"}).add_to(m);

## Grid:

#### Add gridline spacings that we want, covering the whole area:

In [8]:
# here are broad boundaries
gridlons = [-74.5,-73.5]
gridlats = [40.5,50.5]

In [9]:
# extract linear points between these. 
# This controls spacing...
hlines_bigbounds = np.array([[[i,gridlons[0]],[i,gridlons[1]]] for i in np.linspace(gridlats[0],gridlats[1],10001)])
vlines_bigbounds = np.array([[[gridlats[0],i],[gridlats[1],i]] for i in np.linspace(gridlons[0],gridlons[1],1001)])

#### Exclude gridlines that are far from Hillside:

In [10]:
# fit boundaries tightly around Hillside
boundlats = [40.993,41.004]
boundlons = [-73.878,-73.857]

In [11]:
# exclude far-away horizontal lines
hlines = hlines_bigbounds[np.logical_and(hlines_bigbounds[:,0,0] >= boundlats[0], hlines_bigbounds[:,0,0] <= boundlats[1])]
hlines[:,0,1] = boundlons[0]
hlines[:,1,1] = boundlons[1]

In [12]:
# exclude far-away vertical lines
vlines = vlines_bigbounds[np.logical_and(vlines_bigbounds[:,0,1] >= boundlons[0], vlines_bigbounds[:,0,1] <= boundlons[1])]
vlines[:,0,0] = boundlats[0]
vlines[:,1,0] = boundlats[1]

In [13]:
# save all lines for grid
lines = np.vstack([hlines,vlines])

#### Add grid to map -- each gridline is plotted separately:

In [14]:
# loop through gridlines, adding each one
for line in lines:
    pl=folium.PolyLine(locations=line,weight=1)
    m.add_child(pl)

## EpiCollect:

#### Finally, add the EpiCollect data to the map:

In [15]:
# add the EpiCollect data points
# for now, the popup just says that they are points
# later, we could add the EpiCollect information for each point
for i in range(len(lat_lon_pairs)):
    folium.Marker(
        lat_lon_pairs[i],
        popup='epicollect data point'
    ).add_to(m)

# 3) Display the map:

* You can zoom in or out, and drag around the map.  
* You can click the location toggle in the upper-left corner to show your gps location.
* You can click on the EpiCollect points in Hillside Woods to show information about them (although they're currently informationless!)  

In [16]:
m